In [55]:
# %%
import pandas as pd

# Wczytanie danych z pliku CSV
df = pd.read_csv(r'afteretl.csv')

# %%
df.head()

,Unnamed: 0,DR_NO,Date Rptd,DATE OCC,TIME OCC,AREA,AREA NAME,Rpt Dist No,Part 1-2,Crm Cd,...,drug substances,annual earnings,value_divided,Time,Month,Day,Year,Quarter,Day of Week,Vict Age bucket
0,0,10304468,2020-01-08,01/08/2020,2230,3,Southwest,377,2,624,...,Alcohol,118908,22.3,22,1,8,2020,1,Wednesday,35-60
1,1000,200105946,2020-01-29,01/29/2020,720,1,Central,157,2,624,...,Prescription opioids,27044,7.2,7,1,29,2020,1,Wednesday,35-60
2,2000,200205282,2020-01-27,01/27/2020,810,2,Rampart,249,2,956,...,Heroin,127062,8.1,8,1,27,2020,1,Monday,35-60
3,3000,200110265,2020-04-09,03/31/2020,1930,1,Central,163,1,350,...,Cannabis (Marijuana),144944,19.3,19,3,31,2020,1,Thursday,35-60
4,4000,200404736,2020-01-20,01/19/2020,2030,4,Hollenbeck,437,2,627,...,Alcohol,101813,20.3,20,1,19,2020,1,Monday,0-18


In [56]:
df.columns

Index(['Unnamed: 0', 'DR_NO', 'Date Rptd', 'DATE OCC', 'TIME OCC', 'AREA',
       'AREA NAME', 'Rpt Dist No', 'Part 1-2', 'Crm Cd', 'Crm Cd Desc',
       'Mocodes', 'Vict Age', 'Vict Sex', 'Vict Descent', 'Premis Cd',
       'Premis Desc', 'Weapon Used Cd', 'Weapon Desc', 'Status', 'Status Desc',
       'Crm Cd 1', 'Crm Cd 2', 'Crm Cd 3', 'Crm Cd 4', 'LOCATION',
       'Cross Street', 'LAT', 'LON', 'gender of criminal',
       'country of criminal', 'kill count', 'marital status of criminal ',
       'age of criminal', 'previously punished', 'education levels',
       'drug substances', 'annual earnings', 'value_divided', 'Time', 'Month',
       'Day', 'Year', 'Quarter', 'Day of Week', 'Vict Age bucket'],
      dtype='object')

In [57]:
df['count'] = 1
df = df[(df['marital status of criminal '] == 'Divorced') & (df['education levels'] == 'primary')]
df = df.groupby(
    ['Year', 'Quarter','Month','Day','Day of Week','education levels','drug substances','marital status of criminal ','previously punished',]
).agg({'count': 'sum'}).reset_index()

# Zmieniamy nazwę kolumny 'kill count' na 'KillCount'
import numpy as np
# Zmieniamy nazwę kolumny 'kill count' na 'KillCount'
random_values = np.random.uniform(0.001, 0.999, len(df))

df['count'] = df['count'] + random_values


In [58]:
df.head()

,Year,Quarter,Month,Day,Day of Week,education levels,drug substances,marital status of criminal,previously punished,count
0,2020,1,1,2,Wednesday,primary,Cocaine,Divorced,T,2.816907
1,2020,1,1,8,Wednesday,primary,Alcohol,Divorced,T,2.316463
2,2020,1,1,10,Friday,primary,Synthetic cannabinoids,Divorced,F,2.066864
3,2020,1,1,16,Thursday,primary,Alcohol,Divorced,F,2.820862
4,2020,1,2,14,Monday,primary,Cannabis (Marijuana),Divorced,F,2.521964


In [59]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import plotly.graph_objects as go

# Tworzenie ramki danych


# Grupowanie i agregacja danych
df['Date'] = pd.to_datetime(df[['Year', 'Month', 'Day']])
df_grouped = df.groupby('Date')['count'].sum().reset_index()

# Przygotowanie danych do modelu XGBoost
df_grouped['Month'] = df_grouped['Date'].dt.month
df_grouped['Day'] = df_grouped['Date'].dt.day

X = df_grouped[['Month', 'Day']]
y = df_grouped['count']

# Podział danych na zbiór treningowy i testowy
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Dopasowanie modelu XGBoost
model = xgb.XGBRegressor(objective ='reg:squarederror', seed=42)
model.fit(X_train, y_train)

# Prognoza na następny rok
next_year_dates = pd.date_range(start=df_grouped['Date'].max() + pd.DateOffset(days=1), end=df_grouped['Date'].max() + pd.DateOffset(years=1), freq='D')
next_year_X = pd.DataFrame({'Month': next_year_dates.month, 'Day': next_year_dates.day})
forecast = model.predict(next_year_X)

# Wykres szeregu czasowego z prognozą
fig = go.Figure()

# Dodanie rzeczywistych danych
fig.add_trace(go.Scatter(x=df_grouped['Date'], y=df_grouped['count'], mode='lines+markers', name='Actual'))

# Dodanie prognoz
fig.add_trace(go.Scatter(x=next_year_dates, y=forecast, mode='lines+markers', name='Forecast', line=dict(color='red', dash='dash')))

# Aktualizacja układu wykresu
fig.update_layout(title='Przestępcy rozwiedzeni z podstawowym wykształcebniem algorytm xgboost', xaxis_title='Data', 
                  yaxis_title='Liczba zabitych',template="plotly_dark",)

# Wyświetlenie wykresu
fig.show()
# Zapisywanie wykresu do pliku HTML
fig.write_html("forecast_plot3_1.html")


In [60]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.arima.model import ARIMA
import plotly.graph_objects as go



df['Date'] = pd.to_datetime(df[['Year', 'Month', 'Day']])
# Grupowanie i agregacja danych
df_grouped = df.groupby('Date')['count'].sum().reset_index()

# Przygotowanie danych do modelu ARIMA
df_grouped.set_index('Date', inplace=True)

# Dopasowanie modelu ARIMA
model = ARIMA(df_grouped['count'], order=(5, 1, 0))
model_fit = model.fit()

# Prognoza na następny rok
forecast = model_fit.forecast(steps=365)
forecast_dates = pd.date_range(start=df_grouped.index[-1] + pd.DateOffset(days=1), periods=365, freq='D')

# Wykres szeregu czasowego z prognozą
fig = go.Figure()

# Dodanie rzeczywistych danych
fig.add_trace(go.Scatter(x=df_grouped.index, y=df_grouped['count'], mode='lines+markers', name='Actual'))

# Dodanie prognoz
fig.add_trace(go.Scatter(x=forecast_dates, y=forecast, mode='lines+markers', name='Forecast', line=dict(color='red', dash='dash')))

# Aktualizacja układu wykresu
fig.update_layout(title='Przestępcy rozwiedzeni z podstawowym wykształcebniem algorytm ARIMA', xaxis_title='Data',
                   yaxis_title='Liczba incydentów',template="plotly_dark",)

# Wyświetlenie wykresu
fig.show()

# Zapisywanie wykresu do pliku HTML
fig.write_html("forecast_plot3_2.html")


c:\Users\micha\Desktop\WAD_07\Wielowymiarowa_analiza_danych\venv\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning:

A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.

c:\Users\micha\Desktop\WAD_07\Wielowymiarowa_analiza_danych\venv\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning:

A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.

c:\Users\micha\Desktop\WAD_07\Wielowymiarowa_analiza_danych\venv\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning:

A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.

c:\Users\micha\Desktop\WAD_07\Wielowymiarowa_analiza_danych\venv\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: ValueWarning:

No supported index is available. Prediction results will be given with an 

In [61]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from prophet import Prophet
import plotly.graph_objects as go



df['Date'] = pd.to_datetime(df[['Year', 'Month', 'Day']])

# Grupowanie i agregacja danych
df_grouped = df.groupby('Date')['count'].sum().reset_index()

# Przygotowanie danych do modelu Prophet
df_grouped.rename(columns={'Date': 'ds', 'count': 'y'}, inplace=True)

# Dopasowanie modelu Prophet
model = Prophet()
model.fit(df_grouped)

# Prognoza na następny rok
future_dates = model.make_future_dataframe(periods=365)
forecast = model.predict(future_dates)

# Wykres szeregu czasowego z prognozą
fig = go.Figure()

# Dodanie rzeczywistych danych
fig.add_trace(go.Scatter(x=df_grouped['ds'], y=df_grouped['y'], mode='lines+markers', name='Actual'))

# Dodanie prognoz
forecast_future = forecast[forecast['ds'] > df_grouped['ds'].max()]
fig.add_trace(go.Scatter(x=forecast_future['ds'], y=forecast_future['yhat'], mode='lines+markers', name='Forecast', line=dict(color='red', dash='dash')))

# Aktualizacja układu wykresu
fig.update_layout(title='Przestępcy rozwiedzeni z podstawowym wykształcebniem algorytmu Prophet', xaxis_title='Data',
                   yaxis_title='Liczba przestępstw',template="plotly_dark",)

# Wyświetlenie wykresu
fig.show()

# Zapisywanie wykresu do pliku HTML
fig.write_html("forecast_plot3_3.html")


17:46:26 - cmdstanpy - INFO - Chain [1] start processing
17:46:27 - cmdstanpy - INFO - Chain [1] done processing


In [62]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
import plotly.graph_objects as go

# Tworzenie kolumny daty
df['Date'] = pd.to_datetime(df[['Year', 'Month', 'Day']])

# Grupowanie i agregacja danych
df_grouped = df.groupby('Date')['count'].sum().reset_index()

# Przygotowanie danych do modelu Random Forest
df_grouped['Month'] = df_grouped['Date'].dt.month
df_grouped['Day'] = df_grouped['Date'].dt.day

X = df_grouped[['Month', 'Day']]
y = df_grouped['count']

# Podział danych na zbiór treningowy i testowy
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Dopasowanie modelu Random Forest
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Prognoza na następny rok
next_year_dates = pd.date_range(start=df_grouped['Date'].max() + pd.DateOffset(days=1), end=df_grouped['Date'].max() + pd.DateOffset(years=1), freq='D')
next_year_X = pd.DataFrame({'Month': next_year_dates.month, 'Day': next_year_dates.day})
forecast = model.predict(next_year_X)

# Wykres szeregu czasowego z prognozą
fig = go.Figure()

# Dodanie rzeczywistych danych
fig.add_trace(go.Scatter(x=df_grouped['Date'], y=df_grouped['count'], mode='lines+markers', name='Actual'))

# Dodanie prognoz
fig.add_trace(go.Scatter(x=next_year_dates, y=forecast, mode='lines+markers', name='Forecast', line=dict(color='red', dash='dash')))

# Aktualizacja układu wykresu
fig.update_layout(title='Przestępcy rozwiedzeni z podstawowym wykształcebniem przewidywanie Random Forest', 
                  xaxis_title='Data', yaxis_title='Liczba KillCount',template="plotly_dark",)

# Wyświetlenie wykresu
fig.show()

# Zapisywanie wykresu do pliku HTML
fig.write_html("forecast_plot3_4.html")
